In [21]:
# keithley_connect_idn_fix.py
import time
import pyvisa

IPS = {
    "DMM6500 (VRef)":        "192.168.1.103",
    "2450 (Source-Drain)":   "192.168.1.102",
    "2450 (Source-Gate)":    "192.168.1.101",
}

TIMEOUT_MS = 5000
PATTERNS = [
    "TCPIP0::{ip}::inst0::INSTR",   # VXI-11
    "TCPIP0::{ip}::hislip0::INSTR", # HiSLIP
    "TCPIP0::{ip}::5025::SOCKET",   # RAW SOCKET (SCPI)
]

def open_and_idn(rm, resource):
    inst = rm.open_resource(resource)
    inst.timeout = TIMEOUT_MS
    inst.read_termination = "\n"
    inst.write_termination = "\n"
    try:
        try:
            lang = inst.query("SYST:LANG?").strip().upper()
            if lang != "SCPI":
                inst.write("SYST:LANG SCPI")
                time.sleep(0.2)
        except Exception:
            pass
        inst.write("*CLS")
        ident = inst.query("*IDN?").strip()
        return ident
    finally:
        inst.close()

def probe(label, ip, rm):
    print(f"[{label}] probing {ip} ...")
    last_err = None
    for patt in PATTERNS:
        res = patt.format(ip=ip)
        try:
            print(f"  trying {res} ...", end=" ")
            ident = open_and_idn(rm, res)
            print("OK")
            print(f"  *IDN? -> {ident}")
            return
        except Exception as e:
            last_err = e
            print(f"NG ({e})")
    print(f"  all patterns failed. last error: {last_err}")

def main():
    print("=== Keithley connection smoke test ===")
    print("※本スクリプトは *IDN? のみ送信し、出力は一切ONにしません。")
    rm = pyvisa.ResourceManager()
    for label, ip in IPS.items():
        probe(label, ip, rm)
    print("=== Done ===")

if __name__ == "__main__":
    main()



=== Keithley connection smoke test ===
※本スクリプトは *IDN? のみ送信し、出力は一切ONにしません。
[DMM6500 (VRef)] probing 192.168.1.103 ...
  trying TCPIP0::192.168.1.103::inst0::INSTR ... OK
  *IDN? -> KEITHLEY INSTRUMENTS,MODEL DMM6500,04585958,1.7.12b
[2450 (Source-Drain)] probing 192.168.1.102 ...
  trying TCPIP0::192.168.1.102::inst0::INSTR ... OK
  *IDN? -> KEITHLEY INSTRUMENTS,MODEL 2450,04588671,1.7.12b
[2450 (Source-Gate)] probing 192.168.1.101 ...
  trying TCPIP0::192.168.1.101::inst0::INSTR ... OK
  *IDN? -> KEITHLEY INSTRUMENTS,MODEL 2450,04588729,1.7.12b
=== Done ===


In [19]:
# net_diag_keithley.py
import socket

TARGETS = {
    "DMM6500 (VRef)":      "192.168.1.103",
    "2450 (Source-Drain)": "192.168.1.102",
#    "2450 (Source-Gate)":  "192.168.1.101",
}

PORTS = [
    (5025,  b"*IDN?\n"),   # SCPI Socket Server
    (80,    b"GET / HTTP/1.0\r\n\r\n"),  # LXI Web
    (111,   None),         # VXI-11 RPC portmapper
]

def try_port(ip, port, payload=None, timeout=2.0):
    try:
        with socket.create_connection((ip, port), timeout=timeout) as s:
            if payload:
                s.sendall(payload)
                s.settimeout(timeout)
                try:
                    data = s.recv(1024)
                except socket.timeout:
                    data = b""
            else:
                data = b""
        return True, data[:80]
    except Exception as e:
        return False, str(e).encode()

if __name__ == "__main__":
    for label, ip in TARGETS.items():
        print(f"\n[{label}] {ip}")
        for port, payload in PORTS:
            ok, info = try_port(ip, port, payload)
            print(f"  port {port}: {'OPEN' if ok else 'CLOSED'}  info={info!r}")



[DMM6500 (VRef)] 192.168.1.103
  port 5025: OPEN  info=b'KEITHLEY INSTRUMENTS,MODEL DMM6500,04585958,1.7.12b\n'
  port 80: OPEN  info=b'HTTP/1.1 200 OK\r\nContent-Type: text/html\r\nDate: Wed, 15 Oct 2025 19:04:17 GMT\r\nE'
  port 111: OPEN  info=b''

[2450 (Source-Drain)] 192.168.1.102
  port 5025: OPEN  info=b'KEITHLEY INSTRUMENTS,MODEL 2450,04588671,1.7.12b\n'
  port 80: OPEN  info=b'HTTP/1.1 200 OK\r\nContent-Type: text/html\r\nDate: Wed, 15 Oct 2025 19:41:40 GMT\r\nE'
  port 111: OPEN  info=b''


In [22]:
# ============================================
# DMM6500 Vref logger (10 MΩ入力仕様)
# ============================================
import time
import pyvisa

def acquire_vref_dmm6500(
    rm: pyvisa.ResourceManager,
    ip: str,#192.168.1.103
    meas_time_interval: float,#サンプリング間隔
    points: int,
    v_range: float = 1.0,#rangeは1V
    nplc: float = 10.0,
    autozero: str = "ON"
):
    """
    DMM6500で指定間隔ごとにDC電圧(Vref)を取得する。
    Input Impedance = 10 MΩ (標準モード)

    Parameters
    ----------
    rm : pyvisa.ResourceManager
    ip : str                     # DMM6500のIPアドレス
    meas_time_interval : float   # サンプリング間隔 [s]
    points : int                 # サンプル数
    v_range : float              # 電圧レンジ [V]
    nplc : float                 # NPLC (1〜10くらい)
    autozero : str               # "ON" / "OFF"

    Returns
    -------
    list[(float, float)] : [(timestamp_epoch, voltage_V), ...]
    """

    resource = f"TCPIP0::{ip}::inst0::INSTR"
    inst = rm.open_resource(resource)
    inst.timeout = int(max(5000, (meas_time_interval * points + 5) * 1000))
    inst.read_termination = "\n"
    inst.write_termination = "\n"

    try:
        # -----------------------------
        # 設定
        # -----------------------------
        inst.write("*CLS")
        inst.write("SENS:FUNC 'VOLT:DC'")        # DC電圧モード
        inst.write(f"SENS:VOLT:DC:RANG {v_range}") # レンジ 1V固定など
        inst.write(f"SENS:VOLT:DC:NPLC {nplc}")    # 積分時間
        inst.write(f"SYST:AZER {autozero}")        # AutoZero ON/OFF

        # 標準インピーダンス（10 MΩ）
        inst.write("SENS:VOLT:DC:IMP:AUTO ON")

        # 平均化 OFF
        inst.write("SENS:VOLT:DC:AVER:STAT OFF")

        # -----------------------------
        # サンプリング設定
        # -----------------------------
        inst.write("TRAC:CLE")
        inst.write(f"TRAC:POIN {points}")
        inst.write("TRAC:FEED SENS")
        inst.write("TRAC:FEED:CONT NEXT")
        inst.write(f"SAMP:TIM {meas_time_interval}")
        inst.write(f"SAMP:COUN {points}")
        inst.write(f"TRIG:COUN {points}")
        inst.write("TRIG:SOUR IMM")

        # -----------------------------
        # 測定実行
        # -----------------------------
        t_start = time.time()
        inst.write("INIT")
        inst.query("*OPC?")  # 完了待ち

        # -----------------------------
        # データ取得
        # -----------------------------
        raw = inst.query("TRAC:DATA?")
        values = [float(x) for x in raw.strip().split(",") if x.strip()]
        data = [(t_start + i * meas_time_interval, v) for i, v in enumerate(values[:points])]
        return data

    finally:
        try:
            inst.write("TRAC:FEED:CONT NEV")
        except Exception:
            pass
        inst.close()

# --------------------------------------------
# 使用例
# --------------------------------------------
if __name__ == "__main__":
    rm = pyvisa.ResourceManager()
    data = acquire_vref_dmm6500(
        rm,
        ip="192.168.1.103",        # ← DMM6500 の IP
        meas_time_interval=0.2,    # サンプリング間隔 [s]
        points=100,                # サンプル数
        v_range=1.0,               # 1Vレンジ
        nplc=10,                   # ノイズ低減
        autozero="ON"
    )

    for t, v in data[:5]:
        print(f"{t:.3f}, {v:.6f} V")


VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [14]:
# dmm6500_マルチメータ
import pyvisa

# リソースマネージャを作成
rm = pyvisa.ResourceManager()

# DMM6500のIPアドレスを設定
ip = "192.168.1.103"  # 実際のDMM6500のIPアドレスに変更

# 接続
dmm = rm.open_resource(f"TCPIP0::{ip}::inst0::INSTR")
print("Connected to DMM6500.")

# 測定モードをDC電圧に設定
dmm.write("*CLS")  # クリア
dmm.write("SENS:FUNC 'VOLT:DC'")  # DC電圧測定設定
dmm.write("SENS:VOLT:RANG 10")  # 測定レンジ設定 (例: 10V)
dmm.write("SENS:VOLT:INP AUTO")  # 入力インピーダンスを自動設定
dmm.write("SENS:VOLT:NPLC 10")  # 測定の精度設定

# 測定値を取得
voltage = dmm.query("READ?")
print(f"Measured Voltage: {voltage} V")

# 接続を閉じる
dmm.close()


Exception: error creating link: 3

In [23]:
import pyvisa
import time

# リソースマネージャを作成
rm = pyvisa.ResourceManager()

# Keithley2450のIPアドレスを設定（適宜変更してください）
ip = "192.168.1.102"  # Keithley2450のIPアドレス

# 接続
drain_smu = rm.open_resource(f"TCPIP0::{ip}::inst0::INSTR")
print("Connected to Keithley 2450 Drain.")

# 機器情報を確認
drain_smu.write("*IDN?")
response = drain_smu.read()
print("Keithley 2450 Response: ", response)

# 測定の設定
drain_smu.write("*RST")  # 初期化
drain_smu.write("*CLS")  # クリア

# 電圧源として設定
drain_smu.write("SOUR:VOLT:FUNC VOLT")  # 電圧源モード
drain_smu.write("SOUR:VOLT 0.01")  # 10 mVの電圧設定
drain_smu.write("SOUR:VOLT:ILIM 0.001")  # 電流リミット 1 mA設定

# 電流測定設定
drain_smu.write("SENS:FUNC 'CURR'")  # 電流測定モード
drain_smu.write("SENS:CURR:RANG:AUTO ON")  # 電流レンジ自動設定
drain_smu.write("SENS:CURR:NPLC 1")  # NPLC（測定精度）

# 出力をオンにして、測定を開始
drain_smu.write("OUTP ON")

# 少し待ってから電流を測定
time.sleep(10)  # 測定まで1秒待機

# 測定された電流を読み取る
current = drain_smu.query("READ?")
print(f"Measured current: {current} A")

# 出力をオフにする
drain_smu.write("OUTP OFF")

# 接続を閉じる
drain_smu.close()


Connected to Keithley 2450 Drain.
Keithley 2450 Response:  KEITHLEY INSTRUMENTS,MODEL 2450,04588671,1.7.12b

Measured current: 2.917937E-09
 A


In [24]:
# ============================================
# 3機器同時測定システム (測定モード・グラフ追加版)
# DMM6500 (Vref測定) + 2450 (Drain) + 2450 (Gate sweep)
# ============================================
import time
import pyvisa
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from typing import List, Tuple, Dict, Optional
from abc import ABC, abstractmethod
from dataclasses import dataclass
from enum import Enum
import threading


# ============================================
# 測定モード定義
# ============================================
class MeasurementMode(Enum):
    """測定モード"""
    LINEAR = "linear"
    OUTPUT = "output"
    SATURATION = "saturation"


# ============================================
# 測定パラメータデータクラス
# ============================================
@dataclass
class SaturationParams:
    """Saturationモードのパラメータ"""
    vsd_sat: float  # ソースドレイン電圧 [V]
    cycle_num: int  # 測定の繰り返し回数
    vg_sat_initial: float  # ゲート電圧の測定初期値 [V]
    measurement_interval: float  # Vg切り替え後の休み時間 [s]
    vref_sat_initial: float  # Vrefの測定初期値 [V]
    vref_sat_max_change: float  # Vrefの最大(負の場合最小) [V]
    vg_sat_step: float  # ゲート電圧の変化間隔 [V]
    sat_wait_time: float  # 平衡待機時間 [s]
    data_interval: float = 1.0  # データ取得間隔 [s] (デフォルト1秒)


@dataclass
class LinearParams:
    """Linearモードのパラメータ（例）"""
    vds_start: float
    vds_stop: float
    vgs_list: List[float]
    points: int
    interval: float


@dataclass
class OutputParams:
    """Outputモードのパラメータ（例）"""
    vgs_start: float
    vgs_stop: float
    vds_fixed: float
    points: int
    interval: float


# ============================================
# 抽象基底クラス
# ============================================
class InstrumentBase(ABC):
    """測定機器の基底クラス"""
    
    def __init__(self, rm: pyvisa.ResourceManager, ip: str):
        self.rm = rm
        self.ip = ip
        self.inst = None
    
    def connect(self):
        """機器に接続"""
        resource = f"TCPIP0::{self.ip}::inst0::INSTR"
        self.inst = self.rm.open_resource(resource)
        self.inst.read_termination = "\n"
        self.inst.write_termination = "\n"
    
    @abstractmethod
    def configure(self):
        """機器の設定（サブクラスで実装）"""
        pass
    
    def close(self):
        """接続を閉じる"""
        if self.inst is not None:
            self.inst.close()
            self.inst = None


# ============================================
# DMM6500クラス (Vref測定用)
# ============================================
class DMM6500(InstrumentBase):
    """KEITHLEY DMM6500 Digital Multimeter制御クラス"""
    
    def __init__(
        self,
        rm: pyvisa.ResourceManager,
        ip: str,
        v_range: float = 1.0,
        nplc: float = 10.0,
        autozero: str = "ON"
    ):
        super().__init__(rm, ip)
        self.v_range = v_range
        self.nplc = nplc
        self.autozero = autozero
    
    def configure(self):
        if self.inst is None:
            raise RuntimeError("Not connected. Call connect() first.")
    
        self.inst.write("*RST")
        self.inst.write("*CLS")
    
        # DCV測定を選択
        self.inst.write("SENS:FUNC \"VOLT:DC\"")
    
        # レンジとNPLC
        self.inst.write(f"SENS:VOLT:RANG {self.v_range}")
        self.inst.write(f"SENS:VOLT:NPLC {self.nplc}")
    
        # オートゼロ（機能別）
        self.inst.write("SENS:VOLT:AZER ON" if self.autozero.upper()=="ON" else "SENS:VOLT:AZER OFF")
    
        # 入力インピーダンス：AUTO / 10M / 10G から選択可
        # 高インピーダンスが必要なら "SENS:VOLT:INP 10G"
        self.inst.write("SENS:VOLT:INP AUTO")
    
        # 平均化はオフ（必要ならONにしてカウント設定）
        self.inst.write("SENS:VOLT:AVER:STAT OFF")

    
    def read_single(self) -> float:
        """単一電圧測定"""
        if self.inst is None:
            raise RuntimeError("Not connected.")
        result = self.inst.query("READ?")
        return float(result.strip())


# ============================================
# Keithley2450 ドレイン用クラス
# ============================================
class Keithley2450Drain(InstrumentBase):
    """Keithley 2450 SourceMeter (Drain用)"""
    
    def __init__(
        self,
        rm: pyvisa.ResourceManager,
        ip: str,
        source_voltage: float,
        compliance_current: float,
        meas_range: float = None,
        nplc: float = 1.0,
        autozero: str = "ON"
    ):
        super().__init__(rm, ip)
        self.source_voltage = source_voltage
        self.compliance_current = compliance_current
        self.meas_range = meas_range
        self.nplc = nplc
        self.autozero = autozero
    
    def configure(self):
        """電圧ソース・電流測定の設定"""
        if self.inst is None:
            raise RuntimeError("Not connected. Call connect() first.")
        
        self.inst.write("*RST")
        self.inst.write("*CLS")
        
        self.inst.write("SOUR:FUNC VOLT")
        self.inst.write(f"SOUR:VOLT {self.source_voltage}")
        self.inst.write(f"SOUR:VOLT:ILIM {self.compliance_current}")
        
        self.inst.write("SENS:FUNC 'CURR'")
        if self.meas_range is not None:
            self.inst.write(f"SENS:CURR:RANG {self.meas_range}")
        else:
            self.inst.write("SENS:CURR:RANG:AUTO ON")
        self.inst.write(f"SENS:CURR:NPLC {self.nplc}")
        self.inst.write(f"SYST:AZER {self.autozero}")
        
        self.inst.write("OUTP ON")
    
    def set_voltage(self, voltage: float):
        """ソース電圧を設定"""
        if self.inst is None:
            raise RuntimeError("Not connected.")
        self.inst.write(f"SOUR:VOLT {voltage}")
        self.source_voltage = voltage
    
    def read_current(self) -> float:
        """電流測定"""
        if self.inst is None:
            raise RuntimeError("Not connected.")
        result = self.inst.query("READ?")
        return float(result.strip().split(",")[0])
    
    def close(self):
        """接続を閉じる"""
        if self.inst is not None:
            try:
                self.inst.write("OUTP OFF")
            except Exception:
                pass
        super().close()


# ============================================
# Keithley2450 ゲート用クラス
# ============================================
class Keithley2450Gate(InstrumentBase):
    """Keithley 2450 SourceMeter (Gate用)"""
    
    def __init__(
        self,
        rm: pyvisa.ResourceManager,
        ip: str,
        compliance_current: float,
        meas_range: float = None,
        nplc: float = 1.0,
        autozero: str = "ON"
    ):
        super().__init__(rm, ip)
        self.compliance_current = compliance_current
        self.meas_range = meas_range
        self.nplc = nplc
        self.autozero = autozero
        self.current_voltage = 0.0
    
    def configure(self):
        """電圧ソース・電流測定の設定"""
        if self.inst is None:
            raise RuntimeError("Not connected. Call connect() first.")
        
        self.inst.write("*RST")
        self.inst.write("*CLS")
        
        self.inst.write("SOUR:FUNC VOLT")
        self.inst.write(f"SOUR:VOLT:ILIM {self.compliance_current}")
        
        self.inst.write("SENS:FUNC 'CURR'")
        if self.meas_range is not None:
            self.inst.write(f"SENS:CURR:RANG {self.meas_range}")
        else:
            self.inst.write("SENS:CURR:RANG:AUTO ON")
        self.inst.write(f"SENS:CURR:NPLC {self.nplc}")
        self.inst.write(f"SYST:AZER {self.autozero}")
        
        self.inst.write("OUTP ON")
    
    def set_voltage(self, voltage: float):
        """ゲート電圧を設定"""
        if self.inst is None:
            raise RuntimeError("Not connected.")
        self.inst.write(f"SOUR:VOLT {voltage}")
        self.current_voltage = voltage
    
    def read_current(self) -> float:
        """電流測定"""
        if self.inst is None:
            raise RuntimeError("Not connected.")
        result = self.inst.query("READ?")
        return float(result.strip().split(",")[0])
    
    def close(self):
        """接続を閉じる"""
        if self.inst is not None:
            try:
                self.inst.write("OUTP OFF")
            except Exception:
                pass
        super().close()


# ============================================
# リアルタイムグラフ表示クラス
# ============================================
class RealtimePlotter:
    """リアルタイムグラフ表示"""
    
    def __init__(self, mode: MeasurementMode):
        self.mode = mode
        self.fig = None
        self.axes = None
        self.lines = {}
        self.data = {
            'time': [],
            'vref': [],
            'isd': [],
            'vg': [],
        }
        self.setup_plot()
    
    def setup_plot(self):
        """プロット設定"""
        if self.mode == MeasurementMode.SATURATION:
            self.fig, self.axes = plt.subplots(2, 2, figsize=(12, 10))
            self.fig.suptitle('Saturation Mode - Real-time Measurement', fontsize=14)
            
            # Isd vs Vref
            ax1 = self.axes[0, 0]
            ax1.set_xlabel('Vref [mV]')
            ax1.set_ylabel('Isd [A]')
            ax1.set_title('Isd vs Vref')
            ax1.grid(True)
            ax1.invert_xaxis()  # 右側がマイナス
            self.lines['isd_vref'], = ax1.plot([], [], 'b-', linewidth=1.5)
            
            # Isd vs Time
            ax2 = self.axes[0, 1]
            ax2.set_xlabel('Time [s]')
            ax2.set_ylabel('Isd [A]')
            ax2.set_title('Isd vs Time')
            ax2.grid(True)
            self.lines['isd_time'], = ax2.plot([], [], 'r-', linewidth=1.5)
            
            # Log Isd vs Vref
            ax3 = self.axes[1, 0]
            ax3.set_xlabel('Vref [mV]')
            ax3.set_ylabel('log|Isd| [A]')
            ax3.set_title('Log Isd vs Vref')
            ax3.grid(True)
            ax3.invert_xaxis()  # 右側がマイナス
            self.lines['log_isd_vref'], = ax3.plot([], [], 'g-', linewidth=1.5)
            
            # Isd vs Vg
            ax4 = self.axes[1, 1]
            ax4.set_xlabel('Vg [V]')
            ax4.set_ylabel('Isd [A]')
            ax4.set_title('Isd vs Vg')
            ax4.grid(True)
            self.lines['isd_vg'], = ax4.plot([], [], 'm-', linewidth=1.5)
            
            plt.tight_layout()
            plt.ion()
            plt.show()
    
    def update(self, time_val: float, vref: float, isd: float, vg: float):
        """データ更新"""
        self.data['time'].append(time_val)
        self.data['vref'].append(vref * 1000)  # V → mV
        self.data['isd'].append(isd)
        self.data['vg'].append(vg)
        
        if self.mode == MeasurementMode.SATURATION:
            # Isd vs Vref
            self.lines['isd_vref'].set_data(self.data['vref'], self.data['isd'])
            self.axes[0, 0].relim()
            self.axes[0, 0].autoscale_view()
            
            # Isd vs Time
            self.lines['isd_time'].set_data(self.data['time'], self.data['isd'])
            self.axes[0, 1].relim()
            self.axes[0, 1].autoscale_view()
            
            # Log Isd vs Vref
            log_isd = [np.log10(abs(i)) if abs(i) > 1e-15 else -15 for i in self.data['isd']]
            self.lines['log_isd_vref'].set_data(self.data['vref'], log_isd)
            self.axes[1, 0].relim()
            self.axes[1, 0].autoscale_view()
            
            # Isd vs Vg
            self.lines['isd_vg'].set_data(self.data['vg'], self.data['isd'])
            self.axes[1, 1].relim()
            self.axes[1, 1].autoscale_view()
            
            self.fig.canvas.draw()
            self.fig.canvas.flush_events()
    
    def close(self):
        """プロット終了"""
        if self.fig is not None:
            plt.ioff()
            plt.show(block=False)


# ============================================
# 測定制御クラス
# ============================================
class MeasurementController:
    """測定制御クラス"""
    
    def __init__(
        self,
        dmm: DMM6500,
        drain_smu: Keithley2450Drain,
        gate_smu: Keithley2450Gate
    ):
        self.dmm = dmm
        self.drain_smu = drain_smu
        self.gate_smu = gate_smu
        self.plotter: Optional[RealtimePlotter] = None
        self.measurement_data = []
        self.measurement_thread = None
        self.measuring = False
        self.current_vg = 0.0
        self.current_cycle = 1
        self.current_vsd = 0.0
    
    def _measurement_loop(self, t_start: float, data_interval: float):
        """バックグラウンドで常時測定を行うスレッド"""
        next_measure_time = time.time()
        
        while self.measuring:
            current_time = time.time()
            
            if current_time >= next_measure_time:
                t_measure = current_time - t_start
                
                try:
                    # 全データ測定
                    vref = self.dmm.read_single()
                    isd = self.drain_smu.read_current()
                    ig = self.gate_smu.read_current()
                    
                    # データ保存
                    self.measurement_data.append({
                        'time': t_measure,
                        'cycle': self.current_cycle,
                        'vref': vref,
                        'vg': self.current_vg,
                        'isd': isd,
                        'ig': ig,
                        'vsd': self.current_vsd
                    })
                    
                    # グラフ更新
                    if self.plotter is not None:
                        self.plotter.update(t_measure, vref, isd, self.current_vg)
                    
                except Exception as e:
                    print(f"Measurement error: {e}")
                
                # 次の測定時刻を設定
                next_measure_time += data_interval
            
            # 短時間スリープしてCPU負荷を下げる
            time.sleep(0.01)
    
    def measure_saturation(self, params: SaturationParams):
        """Saturationモード測定（1秒ごとのデータ取得）"""
        print("="*60)
        print("Starting SATURATION mode measurement...")
        print(f"Data acquisition interval: {params.data_interval} s")
        print("="*60)
        
        # リアルタイムプロット初期化
        self.plotter = RealtimePlotter(MeasurementMode.SATURATION)
        
        # ドレイン電圧設定
        self.drain_smu.set_voltage(params.vsd_sat)
        self.current_vsd = params.vsd_sat
        print(f"Drain voltage set to {params.vsd_sat} V")
        
        # 測定開始時刻
        t_start = time.time()
        
        # バックグラウンド測定スレッド開始
        self.measuring = True
        self.measurement_thread = threading.Thread(
            target=self._measurement_loop,
            args=(t_start, params.data_interval),
            daemon=True
        )
        self.measurement_thread.start()
        
        try:
            # サイクルループ
            for cycle in range(params.cycle_num):
                self.current_cycle = cycle + 1
                print(f"\n--- Cycle {cycle + 1}/{params.cycle_num} ---")
                
                # Vrefステップ計算
                if params.vref_sat_max_change > params.vref_sat_initial:
                    vref_step = params.vg_sat_step
                    num_steps = int((params.vref_sat_max_change - params.vref_sat_initial) / vref_step) + 1
                else:
                    vref_step = -params.vg_sat_step
                    num_steps = int((params.vref_sat_initial - params.vref_sat_max_change) / params.vg_sat_step) + 1
                
                # Forward sweep: initial → max
                print("Forward sweep...")
                for step in range(num_steps):
                    vg_current = params.vg_sat_initial + step * vref_step
                    
                    # ゲート電圧設定
                    self.gate_smu.set_voltage(vg_current)
                    self.current_vg = vg_current
                    
                    print(f"  Step {step+1}/{num_steps}: Vg={vg_current:.4f}V - Waiting {params.sat_wait_time}s for equilibration...")
                    
                    # 平衡待機（この間もバックグラウンドで測定継続）
                    time.sleep(params.sat_wait_time)
                    
                    # 切り替え後の休み
                    if step < num_steps - 1:  # 最後のステップ以外
                        time.sleep(params.measurement_interval)
                
                # Backward sweep: max → initial
                print("Backward sweep...")
                for step in range(num_steps - 2, -1, -1):
                    vg_current = params.vg_sat_initial + step * vref_step
                    
                    # ゲート電圧設定
                    self.gate_smu.set_voltage(vg_current)
                    self.current_vg = vg_current
                    
                    print(f"  Step {num_steps-step}/{num_steps}: Vg={vg_current:.4f}V - Waiting {params.sat_wait_time}s for equilibration...")
                    
                    # 平衡待機（この間もバックグラウンドで測定継続）
                    time.sleep(params.sat_wait_time)
                    
                    # 切り替え後の休み
                    if step > 0:  # 最後のステップ以外
                        time.sleep(params.measurement_interval)
        
        finally:
            # 測定スレッド停止
            self.measuring = False
            if self.measurement_thread is not None:
                self.measurement_thread.join(timeout=2.0)
        
        print("\n" + "="*60)
        print("SATURATION mode measurement completed.")
        print(f"Total data points: {len(self.measurement_data)}")
        print("="*60)
    
    def save_data(self, filename: str = "measurement_saturation.txt"):
        """データ保存"""
        if not self.measurement_data:
            print("No data to save.")
            return
        
        print(f"\nSaving data to {filename}...")
        
        with open(filename, "w") as f:
            # ヘッダー
            f.write("Time\tCycle\tVref\tVg\tIsd\tIg\tVsd\n")
            
            # データ
            for data in self.measurement_data:
                f.write(f"{data['time']:.7E}\t")
                f.write(f"{data['cycle']}\t")
                f.write(f"{data['vref']:.7E}\t")
                f.write(f"{data['vg']:.7E}\t")
                f.write(f"{data['isd']:.7E}\t")
                f.write(f"{data['ig']:.7E}\t")
                f.write(f"{data['vsd']:.7E}\n")
        
        print(f"Data saved successfully. Total points: {len(self.measurement_data)}")
    
    def cleanup(self):
        """クリーンアップ"""
        if self.plotter is not None:
            self.plotter.close()


# ============================================
# 使用例：Saturationモード測定
# ============================================
if __name__ == "__main__":
    rm = pyvisa.ResourceManager()
    
    # ========================================
    # 機器インスタンス作成
    # ========================================
    dmm = DMM6500(
        rm=rm,
        ip="192.168.1.103",
        v_range=1.0,
        nplc=10,
        autozero="ON"
    )
    
    drain_smu = Keithley2450Drain(
        rm=rm,
        ip="192.168.1.104",
        source_voltage=5.0,
        compliance_current=0.1,
        meas_range=0.1,
        nplc=1.0,
        autozero="ON"
    )
    
    gate_smu = Keithley2450Gate(
        rm=rm,
        ip="192.168.1.105",
        compliance_current=0.001,
        meas_range=0.001,
        nplc=1.0,
        autozero="ON"
    )
    
    # ========================================
    # Saturationモードパラメータ設定
    # ========================================
    sat_params = SaturationParams(
        vsd_sat=5.0,                    # Vds = 5V
        cycle_num=2,                    # 2サイクル
        vg_sat_initial=0.08,            # Vg初期値 = 0.08V
        measurement_interval=0.1,       # Vg切り替え後の休み = 0.1s
        vref_sat_initial=0.08,          # Vref初期値 = 0.08V
        vref_sat_max_change=-0.8,       # Vref最大変化 = -0.8V
        vg_sat_step=0.02,               # Vgステップ = 0.02V
        sat_wait_time=5.0,              # 平衡待機時間 = 5.0s
        data_interval=1.0               # データ取得間隔 = 1.0s
    )
    
    try:
        # ========================================
        # 接続と設定
        # ========================================
        print("Connecting to instruments...")
        dmm.connect()
        dmm.configure()
        
        drain_smu.connect()
        drain_smu.configure()
        
        gate_smu.connect()
        gate_smu.configure()
        
        print("All instruments configured.\n")
        
        # ========================================
        # 測定実行
        # ========================================
        controller = MeasurementController(dmm, drain_smu, gate_smu)
        controller.measure_saturation(sat_params)
        
        # ========================================
        # データ保存
        # ========================================
        controller.save_data("measurement_saturation.txt")
        
        # グラフを表示したまま待機
        input("\nPress Enter to close the graphs and exit...")
        
    finally:
        # ========================================
        # クリーンアップ
        # ========================================
        print("\nClosing connections...")
        controller.cleanup()
        dmm.close()
        drain_smu.close()
        gate_smu.close()
        print("All connections closed.")

Connecting to instruments...

Closing connections...


NameError: name 'controller' is not defined